In [3]:
const file = Deno.readFileSync("sample/profile.jpg");
//https://yasoob.me/posts/understanding-and-writing-jpeg-decoder-in-python/
function decode(file) {
	const dataView = new DataView(file.buffer);
	let i = 0;

	if (dataView.getUint16(i) !== 0xffd8) {
		throw new Error("Incorrect file identifier");
	}
	i += 2;


	while (true) {
		const marker = dataView.getUint16(i);
		i += 2;

		if (marker === 0xffd9) { //end
			break;
		}

		const length = dataView.getUint16(i); //length includes itself
		i += 2;

		switch(marker) {
			case 0xffe0: {
				decodeApplicationHeader(dataView, i);
				i += length - 2;
				break;
			}
			case 0xffc4: { //huffman
				decodeHuffman(dataView, i);
				i += length - 2;
				break;
			}
			case 0xffdb: {
				decodeQuantizationTable(dataView, i);
				i += length - 2;
				break;
			}
			case 0xffc0: {
				decodeFrame(dataView, i);
				i += length - 2;
				break;
			}
			case 0xffda: {
				decodeScan(dataView, i);
				i += length - 2;
				break;
			}
			default: {
				return "dunno"; //dunno
			}
		}
	}
	return "ok";
}

function decodeApplicationHeader(dataView, index) {
	const identifier = getCString(dataView, index);
	index += identifier.length + 1;
	const [majorVersion, minorVersion] = getBytes(dataView, index, 2);
	index += 2;
	const units = dataView.getUint8(index);
	index += 1;
	const density = getShorts(dataView, index, 2);
	index += 4;
	const thumbnail = getBytes(dataView, index, 2);
	index += 2;
	console.log({
		majorVersion,
		minorVersion,
		units,
		density,
		thumbnail
	})
}

function decodeQuantizationTable(dataView, index){
	console.log("table")
}

function decodeFrame(dataView, index){
	console.log("frame")
}

function decodeScan(dataView, index){
	console.log("scan")
}

function decodeHuffman(dataView, index) {
	const header = dataView.getUint8(index);
	index += 1;

	const lengths = new Array(16);
	for (let i = 0; i < 16; i++) {
		lengths[i] = dataView.getUint8(index + i);
	}

	const elements = new Array(16);
	for (let i = 0; i < 16; i++) {
		elements[i] = getBytes(dataView, index, lengths[i]);
		index += lengths[i];
	}

	console.log({
		header,
		lengths,
		elements,
	});
	console.log([lengths.length, elements.length]);
}

function getBytes(dataView, index, length) {
	const bytes = new Array(length);
	for (let i = 0; i < length; i++) {
		bytes[i] = dataView.getUint8(index + i);
	}
	return bytes;
}

function getShorts(dataView, index, length) {
	const bytes = new Array(length);
	for (let i = 0; i < length; i += 2) {
		bytes[i] = dataView.getUint16(index + i);
	}
	return bytes;
}

function getCString(dataView, index) {
	let string = "";
	let i = 0;
	while (true) {
		const c = dataView.getUint8(index + i);
		if (c === 0) return string;
		string += String.fromCharCode(c);
		i++;
	}
}

function arrayEqual(arrA, arrB) {
	return arrA.every((x, i) => x === arrB[i]);
}


const jpg = decode(file);

{
  majorVersion: 1,
  minorVersion: 1,
  units: 0,
  density: [ 1, <1 empty item> ],
  thumbnail: [ 0, 0 ]
}
table
table
frame
{
  header: 0,
  lengths: [
    0, 2, 2, 3, 1, 1,
    1, 0, 0, 0, 0, 0,
    0, 0, 0, 0
  ],
  elements: [
    [],       [ 0, 2 ],
    [ 2, 3 ], [ 1, 1, 1 ],
    [ 0 ],    [ 0 ],
    [ 0 ],    [],
    [],       [],
    [],       [],
    [],       [],
    [],       []
  ]
}
[ 16, 16 ]
{
  header: 16,
  lengths: [
    0, 2, 1, 3, 2, 4,
    5, 2, 4, 4, 3, 4,
    8, 5, 5, 1
  ],
  elements: [
    [],
    [ 0, 2 ],
    [ 1 ],
    [ 3, 2, 4 ],
    [ 5, 2 ],
    [ 4, 4, 3, 4 ],
    [ 8, 5, 5, 1, 1 ],
    [ 2, 3 ],
    [ 0, 4, 17, 5 ],
    [ 33, 6, 18, 49 ],
    [ 65, 7, 19 ],
    [ 34, 81, 97, 20 ],
    [
      113,  8, 50, 129,
      145, 21, 35,  66
    ],
    [ 161, 82, 177, 193, 51 ],
    [ 98, 209, 225, 9, 22 ],
    [ 23 ]
  ]
}
[ 16, 16 ]
{
  header: 1,
  lengths: [
    0, 2, 3, 1, 1, 1,
    0, 0, 0, 0, 0, 0,
    0, 0, 0, 0
  ],
  elements: [
    [],          [ 